Students:

- ...
- ...
- ...

# Deep Learning - Lab Exercise 2

**WARNING:** you must have finished the first exercise before this one as you will re-use parts of the code.

In the first lab exercise, we built a simple linear classifier.
Although it can give reasonable results on the MNIST dataset (~92.5% of accuracy), deeper neural networks can achieve more the 99% accuracy.
However, it can quickly become really impracical to explicitly code forward and backward passes.
Hence, it is useful to rely on an auto-diff library where we specify the forward pass once, and the backward pass is automatically deduced from the computational graph structure.

In this lab exercise, we will build a small and simple auto-diff lib that mimics the autograd mechanism from Pytorch (of course, we will simplify a lot!)


In [ ]:
# import libs that we will use
import os
import numpy as np
import matplotlib.pyplot as plt
import math

# To load the data we will use the script of Gaetan Marceau Caron
# You can download it from the course webiste and move it to the same directory that contains this ipynb file
import dataset_loader

%matplotlib inline

# Data

In [ ]:
# Download mnist dataset 
if("mnist.pkl.gz" not in os.listdir(".")):
    pass
    # this link doesn't work any more,
    # seach on google for the file "mnist.pkl.gz"
    # and download it
    #!wget http://deeplearning.net/data/mnist/mnist.pkl.gz

# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"

In [ ]:
# load the 3 splits
train_data, dev_data, test_data = dataset_loader.load_mnist(mnist_path)

In [ ]:
train_data[0].shape

In [ ]:
index = 900
label = train_data[1][index]
picture = train_data[0][index]

print("label: %i" % label)
plt.imshow(picture.reshape(28,28), cmap='Greys')

# Computation nodes

Instead of directly manipulating numpy arrays, we will manipulate abstraction that contains:
- a value (i.e. a numpy array)
- a bool indicating if we wish to compute the gradient with respect to the value
- the gradient with respect to the value
- the operation to call during backpropagation

There will be two kind of nodes:
- Tensor: a generic computation node
- Parameter: a computation node that is used to store parameters of the network. Parameters are always leaf nodes, i.e. they cannot be build from other computation nodes.

Our implementation of the backward pass will be really simple and incorrect in the general case (i.e. won't work with computation graph with loops).
We will just apply the derivative function for a given tensor and then call the ones of its antecedents, recursively.
This simple algorithm is good enough for this exercise.

Note that a real implementation of backprop will store temporary values during forward that can be used during backward to improve computation speed. We do not do that here.

In [ ]:
class Tensor:
    def __init__(self, data, require_grad=False):
        # test type of data: should be np array
        if isinstance(data, float):
            data = np.array([data,])
        if type(data) != np.ndarray:
            raise RuntimeError("Input should be a numpy array")

        # store data for this tensor
        self.data = data
        self.require_grad = require_grad
        
        # this values should be set to enable autograd!
        self.gradient = None
        self.d = None
        self.backptr = None
        
    def zero_grad(self):
        """
        Set the gradient of thie tensor to 0
        """
        if self.require_grad:
            self.gradient = np.zeros_like(self.data)
            
    def accumulate_gradient(self, gradient):
        """
        Accumulte gradient for this tensor
        """
        if gradient.shape != self.data.shape:
            print(f"gradient.shape:{gradient.shape}, self.data.shape:{self.data.shape}")
            raise RuntimeError("Invalid gradient dimension")

        if self.gradient is None:
            self.gradient = np.copy(gradient)
        else:
            self.gradient += gradient
            
    def backward(self, g=None):
        """
        The backward pass!
        If g != None, then g is the gradient for the current node.
        i.e. g will be != None only for the loss output.
        
        You should call the function stored in self.d with correct arguments,
        and then recursively call the backward methods of tensors in the backptr list if:
        1. they require a gradient
        2. they are of type Tensor: check with isinstance(o, Tensor)
        """
        
        if not self.require_grad:  # stop right now if this node does not require a gradient
            return
        if g is not None:
            if isinstance(g, float):
                g = np.array([g])
            if type(g) != np.ndarray:
                raise RuntimeError("Gradient should be a numpy array")
            if g.shape != self.data.shape:
                print(f"g.shape:{g.shape}, self.data.shape:{self.data.shape}")
                raise RuntimeError("Gradient of different size than the value!")
                
            self.gradient = g
        
        self.d(self.backptr,self.gradient) #calculates the gradient of the Tensors in backptr with respect to the derivative d
        for i in self.backptr: #iterates through backptr
            #print(f"shape of new backptr item: {i.data.shape}")
            if i.require_grad and not isinstance(i, Parameter) and i.backptr is not None: #checking if the Tensors requires grad, in not a Parameter and if its backptr is not None (to avoid calling backward on the backptr of the input Tensor)
                i.backward(g = i.gradient) #calls backward recursively on i
        
        
        
    
    
class Parameter(Tensor):
    """
    This class will be used to store parameters of the network only!
    """
    def __init__(self, data, name="unamed"):
        super().__init__(data, require_grad=True)
        self.name = name
        
    def backward(self):
        raise RuntimeError("You cannot backprop from a Parameter node")

# Functions

Functions manipulate tensors and build the required information for autograd.
A function returns a Tensor that should have require_grad = True if at least of the arguments require a gradient.

In [ ]:
def any_require_grad(l):
    """
    Input:
    - l: an iterable (e.g. a list)
    Ouput:
    - True if any tensor in the input requires a gradient
    """
    return any(t.require_grad for t in l)

In [ ]:
# Here is an exemple with the ReLU
def relu(x):
    v = np.maximum(0, x.data)
    
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_relu
    output.backptr = [x]
    
    return output

def backward_relu(backptr, g):
    x, = backptr
    
    # the gradient is accumulated in the arguments only if required
    if x.require_grad:
        x.accumulate_gradient(g * (x.data > 0))

In [ ]:
def tanh(x):
    v = (1 - np.exp(-2*x.data))/(1 + np.exp(-2*x.data))
    
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_tanh
    output.backptr = [x]
    
    return output
    

def backward_tanh(backptr, g):
    x, = backptr
    
    if x.require_grad:
        x.accumulate_gradient(g * (1 - ((1 - np.exp(-2*x.data))/(1 + np.exp(-2*x.data)))**2))

Next, we implement the affine transform operation.
You can reuse the code from the first lab exercise, with one major difference: you have to compute the gradient with respect to x too!

In [ ]:
def affine_transform(W, b, x):
    v = W.data@x.data + b.data
    
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_affine_transform
    output.backptr = [W, b, x]
    
    return output

def backward_affine_transform(backptr, g):
    W, b, x = backptr
    # the gradient is accumulated in the arguments only if required
    if W.require_grad:
        W.accumulate_gradient(((np.ones(W.data.shape)*x.data).T*g).T) #derivative with respect to W
    if b.require_grad:
        b.accumulate_gradient(g*np.ones(b.data.shape[0])) #derivative with respect to b
    if isinstance(x, Tensor) and x.require_grad:
        # (10, 100) (10,) (100,) (10,)
        #print(W.data.shape, b.data.shape, x.data.shape, g.shape)
        x.accumulate_gradient(g@W.data) #derivative with respect to x

In [ ]:
# we use an underscore because this function does not manipulate tensors:
# it is exactly the same as in the previous exercise
def _softmax(x):
    y = np.exp(x - np.max(x))
    return y/np.sum(y,axis=0)
    

def nll(x, gold):
    v = -x.data[gold.data] + np.max(x.data) + np.log(np.sum(np.exp(x.data - np.max(x.data))))
    
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_nll
    output.backptr = [x, gold] #gold is stored in backptr because it is needed when we call backward_nll
    
    return output
    

def backward_nll(backptr, g):
    x, gold = backptr
    
    if x.require_grad:
        g_x = _softmax(x.data)
        g_x[gold.data] -= 1
        #print(f"g_x{g_x}")
        x.accumulate_gradient(g_x)

# Module

Neural networks or parts of neural networks will be stored in Modules.
They implement method to retrieve all parameters of the network and subnetwork.

In [ ]:
class Module:
    def __init__(self):
        raise NotImplemented("")
        
    def parameters(self):
        ret = []
        for name in dir(self):
            o = self.__getattribute__(name)

            if type(o) is Parameter:
                ret.append(o)
            if isinstance(o, Module) or isinstance(o, ModuleList):
                ret.extend(o.parameters())
        return ret

# if you want to store a list of Parameters or Module,
# you must store them in a ModuleList instead of a python list,
# in order to collect the parameters correctly
class ModuleList(list):
    def parameters(self):
        ret = []
        for m in self:
            if type(m) is Parameter:
                ret.append(m)
            elif isinstance(m, Module) or isinstance(m, ModuleList):
                ret.extend(m.parameters())
        return ret

# Initialization and optimization

In [ ]:
def zero_init(b):
    b[:] = 0.

def glorot_init(W):
    W[:] = np.random.uniform(-np.sqrt(6)/np.sqrt(W.shape[0] + W.shape[1]), np.sqrt(6)/np.sqrt(W.shape[0] + W.shape[1]), W.shape)
    
# Look at slides for the formula!
def kaiming_init(W):
    W[:] = np.random.uniform(-np.sqrt(6)/np.sqrt(W.shape[1]), np.sqrt(6)/np.sqrt(W.shape[1]), W.shape)

In [ ]:
# simple gradient descent optimizer
class SGD:
    def __init__(self, params, lr=0.1):
        self.params = params
        self.lr = lr
        
    def step(self):
        for p in self.params:
            p.data[:] = p.data - self.lr * p.gradient
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

In [ ]:
# simple gradient descent optimizer
class MomentumSGD:
    def __init__(self, params, lr=0.1, mu= 0.9):
        self.params = params
        self.lr = lr
        self.velocity = []
        self.mu = mu
        
    def reset_velocity(self):
        self.velocity = []
        for p in self.params:
            self.velocity.append(np.zeros_like(p.data))
        
    def step(self):
        for i,p in enumerate(self.params):
            self.velocity[i] = self.mu*self.velocity[i] + self.lr*p.gradient
            p.data[:] = p.data - self.velocity[i]
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

# Networks and training loop

We first create a simple linear classifier, similar to the first lab exercise.

In [ ]:
class LinearNetwork(Module):
    def __init__(self, dim_input, dim_output):
        # build the parameters
        self.W = Parameter(np.ndarray((dim_output, dim_input)), name="W") #W paramters of the single layer
        self.b = Parameter(np.ndarray((dim_output,)), name="b") #b parameters of the single layer
        
        self.init_parameters()
        
    def init_parameters(self):
        # init parameters of the network (i.e W and b)
        glorot_init(self.W.data)
        zero_init(self.b.data)
        
    def forward(self, x):
        return  affine_transform(self.W, self.b, x)

We will train several neural networks.
Therefore, we encapsulate the training loop in a function.

**warning**: you have to call optimizer.zero_grad() before each backward pass to reinitialize the gradient of the parameters!

In [ ]:
def training_loop(network, optimizer, train_data, dev_data,momentum, n_epochs=10):
    #create the lists to stores the loss and accuracy of each epoch
    list_loss_train = [] 
    list_loss_test = []
    list_acc_train = []
    list_acc_test = []
    for ep in range(0,n_epochs): #iterate through the epochs
        indexes = np.arange(train_data[0].shape[0])
        np.random.shuffle(indexes) #shuffles the train dataset
        if momentum:
            optimizer.reset_velocity();
        ep_loss_train = 0
        ep_loss_test = 0
        ep_acc_train = 0
        ep_acc_test = 0
        for i in range(train_data[0].shape[0]): #iterates through the dataset
            optimizer.zero_grad() #sets the gradients to 0
            data = Tensor(train_data[0][indexes[i]], True) #loads the input image
            label = Tensor(np.array([train_data[1][indexes[i]]])) #loads its label
            out = network.forward(data) #gets the output of the network
            loss = nll(out, label) #calculates the loss
            #updates loss/accuracy
            ep_loss_train += loss.data
            ep_acc_train += int(np.argmax(out.data) == label.data)
            loss.backward() #propagates the gradient
            optimizer.step() #updates the parameters
        for i in range(dev_data[0].shape[0]): #iterates through the test set
            data = Tensor(dev_data[0][i]) #loads the input image with no require_grad
            label = Tensor(np.array([dev_data[1][i]])) #loads its label
            out = network.forward(data) #gets the output of the network 
            loss = nll(out, label) #calculates the loss
            #updates loss/accuracy
            ep_loss_test += loss.data
            ep_acc_test += int(np.argmax(out.data) == label.data)
        #adds the loss/accuracy of the epoch to the lists
        list_loss_train.append(ep_loss_train/train_data[0].shape[0])
        list_loss_test.append(ep_loss_test/dev_data[0].shape[0])
        list_acc_train.append(ep_acc_train/train_data[0].shape[0])
        list_acc_test.append(ep_acc_test/dev_data[0].shape[0])
        #prints epoch's loss/accuracy
        print(f"---------------------------------------- Epoch {ep} ----------------------------------------")
        print(f"loss_train {round(ep_loss_train[0]/train_data[0].shape[0],3)}, acc_train {round(ep_acc_train/train_data[0].shape[0],3)}")
        print(f"loss_test {round(ep_loss_test[0]/dev_data[0].shape[0],3)}, acc_test {round(ep_acc_test/dev_data[0].shape[0],3)}")
    return list_loss_train, list_loss_test, list_acc_train, list_acc_test
            

In [ ]:
dim_input = 28*28
dim_output = 10

n_epochs = 10

momentum = False

network = LinearNetwork(dim_input, dim_output)

optimizer = MomentumSGD(network.parameters(), 0.001) if momentum else SGD(network.parameters(), 0.001)

list_loss_train, list_loss_test, list_acc_train, list_acc_test = training_loop(network, optimizer, train_data, dev_data, n_epochs=n_epochs, momentum = momentum)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (10,5))

ax[0].plot(np.arange(0,n_epochs,1), list_loss_train, label = 'Train Set')
ax[0].plot(np.arange(0,n_epochs,1), list_loss_test, label = 'Test Set')
ax[0].set_title('Loss/Epoch')
ax[0].legend()
ax[1].plot(np.arange(0,n_epochs,1), list_acc_train, label = 'Train Set')
ax[1].plot(np.arange(0,n_epochs,1), list_acc_test, label = 'Test Set')
ax[1].set_title('Accuracy/Epoch')
ax[1].legend()

After you finished the linear network, you can move to a deep network!

In [ ]:
class DeepNetwork(Module):
    def __init__(self, dim_input, dim_output, hidden_dim, n_layers, tanh=False):
        self.W = ModuleList() #stores the W parameters of each layer
        self.b = ModuleList() #stores the b parameters of each layer
        self.tanh = tanh
        
        for i in range(n_layers): #iterates through the hidden layers
            dim_in = dim_input if i == 0 else hidden_dim #the input dimension is dim_input for the first hidden layer and hidden_dim for all the other hidden layers
            dim_out = hidden_dim #the output dimension is always hidden_dim here
            self.W.append(Parameter(np.ndarray((dim_out, dim_in)), name="W")) #add W to the list
            self.b.append(Parameter(np.ndarray((dim_out,)), name="b")) #add b to the list
        
        #creates the final layer
        self.output_proj = Parameter(np.ndarray((dim_output, hidden_dim)), name="W") #W parameters of the final layer
        self.output_bias = Parameter(np.ndarray((dim_output,)), name="b") #b parameters of the final layer
        
        self.init_parameters()
        
    def init_parameters(self):
        for i in self.W:
            glorot_init(i.data)
        for i in self.b:
            zero_init(i.data)
        glorot_init(self.output_proj.data)
        zero_init(self.output_bias.data)
    def forward(self, x):
        for i in range(len(self.W)): #going forward through all the hidden layers
            if self.tanh: #checking the activation function
                x = tanh(affine_transform(self.W[i],self.b[i],x))
            else:
                x = relu(affine_transform(self.W[i],self.b[i],x))
        return affine_transform(self.output_proj,self.output_bias,x) #forward in the last layers with no activation function
            

In [ ]:
dim_input = 28*28
dim_output = 10
    
n_epochs=10

momentum = False

network = DeepNetwork(dim_input, dim_output, 100, 2)

optimizer = MomentumSGD(network.parameters(), 0.001) if momentum else SGD(network.parameters(), 0.001)

list_loss_train, list_loss_test, list_acc_train, list_acc_test = training_loop(network, optimizer, train_data, dev_data, n_epochs=n_epochs, momentum = momentum)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (10,5))

ax[0].plot(np.arange(0,n_epochs,1), list_loss_train, label = 'Train Set')
ax[0].plot(np.arange(0,n_epochs,1), list_loss_test, label = 'Test Set')
ax[0].set_title('Loss/Epoch')
ax[0].legend()
ax[1].plot(np.arange(0,n_epochs,1), list_acc_train, label = 'Train Set')
ax[1].plot(np.arange(0,n_epochs,1), list_acc_test, label = 'Test Set')
ax[1].set_title('Accuracy/Epoch')
ax[1].legend()

## Bonus

You can try to implement a momentum SGD optimizer! Note that you have to keep track of the velocity for each parameter in the optimizer.
